# Introduction to Artificial Intelligence
Laboratory 1



Exercise 1 Route searching 
1.	Create a set of cities (as points) with coordinates x, y on a plane with height as z coordinate. The cost of going from city A to city B is equal to the Euclidean distance between two cities, if there exists a road. You should define scenarios according to two criteria: 
a.	There are all the direct connections / c.a. 80% of possible connections
b.	The problem is symmetrical / asymmetrical (in asymmetrical – going up is height +10%, going down: -10%)
You should choose the coordinates randomly from the range <-100, 100> for x,y and <0, 50> for z.
2.	Represent the created map as a weighted (directed) graph, where cities are the nodes and roads are the edges of the graph.
3.	In the created scene, solve the traveling salesman problem: The salesman starts from a chosen city and has to visit every city exactly once before returning to the starting city. The goal is to find a path with the lowest cost.
In the problem, we define state as a partial or full path from the starting city and the corresponding state. You should represent the search problem in a form of state tree.
a.	Implement a full search of the tree, using BFS and DFS methods.
b.	Approximate the solution using greedy search (NN and Dijkstra)
c.	Solve/approximate the solution using A* with inadmissible/admissible heuristics
d.	Approximate the solution using ACO algorithm
4.	Test each algorithm, in each scenario, for n=5…20 cities, in terms of the found path cost, time and memory consumption.


In [160]:
import math

def euclidean_distance(a,b)->float:  
    """ 
    points as a 
    tuple or list for example a = (1,2,3) , b = (4,5,6)
    """
    import math
    dimension = len(a)
    if (dimension !=len(b)):
        print("Dimension missmatch")
        return 0 
    dist = 0
    for i in range(dimension):
        dist += math.pow(a[i]-b[i],2)
    return math.sqrt(dist)
    
    
    
def euclidean_distance_assymetrical(a,b)->float:  
    """ 
    points as a 
    tuple or list for example a = (1,2,3) , b = (4,5,6)
    """
    
    dimension = len(a)
    if (dimension !=len(b)):
        print("Dimension missmatch")
        return 0 
    dist = 0
    for i in range(dimension-1): # let's assume last dimesion as high 
        dist += math.pow(a[i]-b[i],2)
        
    if ((a[-1]-b[-1]) > 0):   # a is higher than b so distance from a to b 
        dist += math.pow(0.9*(a[-1]-b[-1]),2)
    else:
        dist += math.pow(1.1*(a[-1]-b[-1]),2)
    
    return math.sqrt(dist)


In [161]:
import random
class CitiesMap():
    def __init__(self,n,seed = None):
        """
        initialize with number of cities to include in graph 
        """    
        
        if seed is not None:
            random.seed(seed)
        self.seed = seed
        self.n = n 
        self.cities = []
        self.routes = [[0 for _ in range(n)] for _ in range(n)]
        # we can represent routes between cities as a matrix with n x n shape 
        for i in range(n):
            self.cities.append((random.randint(-101,101),random.randint(-101,101),random.randint(0,51)))

        # now let's make sure that we don't have duplicated cities so we'll get derired number of unique cities
        
        len_check = len(set(self.cities))
        
        while (len_check != n):
            self.cities.append((random.randint(-101,101),random.randint(-101,101),random.randint(0,51)))
            len_check = len(set(self.cities))
        self.cities = list(set(self.cities))
    
    
    def calculate_distance(self,symmetrical: bool = True , connections: float = 1):
        """
        Fill matrix with distnaces between city i and j  0 < i,j < n with 0 on main diagonal , conenctions 1 means all cities will be connected
        0.5 only half of them and so on 
        
        """
             
        if (symmetrical):
            if (connections==1):   
                for i in range(self.n):
                    for j in range(i+1 , self.n):  # to not double calcualte
                        
                        distance = euclidean_distance(self.cities[i], self.cities[j])
                    
                        # Store the route only once, as it is undirected
                        self.routes[i][j] = distance
                        self.routes[j][i] = distance
                return self.routes
            else:
                # in this case we nned to delete number of connections that equals to connections * (n^2-n)/2 
                
                n_to_delete = int(round((1 - connections) * ((self.n**2 - self.n) / 2),0))

                indexes_list = []    
                                
                for i in range(self.n):
                    for j in range(i+1 , self.n):  # to not double calcualte
                        
                        indexes_list.append((i,j))
                        
                        distance = euclidean_distance(self.cities[i], self.cities[j])
                    
                        # Store the route only once, as it is undirected
                        self.routes[i][j] = distance
                        self.routes[j][i] = distance
            
                indexes_to_delete = random.sample(indexes_list,n_to_delete)
                
                for i in indexes_to_delete:
                    # in the list that we iterate over we have pair of matrix indexes when let's say i means position with 0 index in matrix rows, j column
                    self.routes[i[0]][i[1]] = 0     # deletes i,i
                    self.routes[i[1]][i[0]] = 0        # deletes j,i 
                    
                    
                
                
                return self.routes
                
                
                
        else: 
            if (connections==1):  
                for i in range(self.n):
                    for j in range(self.n):
                        self.routes[i][j] = euclidean_distance_assymetrical(self.cities[i], self.cities[j])
                        
                return self.routes  
            else:
                # in this case we nned to delete number of connections that equals to connections * (n^2-n) # assymetrical 
                
                n_to_delete = int(round((1 - connections) * ((self.n**2 - self.n) / 2),0))
                indexes_list = []    
                                
                for i in range(self.n):
                    for j in range(self.n):
                        # however with connections to delete we'll keep approach from previous one as I understand that lack connections means no road between two points 
                        # no matter how long 
                        
                        self.routes[i][j] = euclidean_distance_assymetrical(self.cities[i], self.cities[j])
                        
                        # adds to indexes base only ones from above main diagonal 
                        if (j>i):
                            indexes_list.append((i,j))
                    
                    
                        
                indexes_to_delete = random.sample(indexes_list,n_to_delete)
                
                
                for i in indexes_to_delete:
                    # in the list that we iterate over we have pair of matrix indexes 
                    self.routes[i[0]][i[1]] = 0     # deletes i,i
                    self.routes[i[1]][i[0]] = 0
                    
                    
                
                
                return self.routes
                
                
    
    


In [162]:
from queue import PriorityQueue
from sys import maxsize
class RouteSearch:
    def __init__(self,cities_map : CitiesMap):
        self.nodes = cities_map.cities
        self.size = cities_map.n
        self.edges = cities_map.routes
        # self.states = []
        self.max = maxsize
    def dfs_all_paths(self, starting_node: int):
        """
        Perform DFS to explore all possible paths, both complete and incomplete.
        
        Parameters:
        starting_node : int - the node from which to start the search
        
        Returns:
        list : A list of paths. Path can be incomplete or complete.
        """
        # Stack for DFS
        stack = [[starting_node]]
        all_paths = []
    
        while stack:
            path = stack.pop()
            last_node = path[-1]
            
            
            # Record the current (incomplete or complete) path and its cost
            all_paths.append(path)

            # If the path includes all cities, check if it can return to the start
            if len(path) == self.size:
                return_to_start_cost = self.edges[last_node][starting_node]
                if return_to_start_cost != 0:     # to make sure that we can return to start from there
                    # Record the completed cycle (path returns to the start)
                    all_paths.append(path + [starting_node])

            # Explore all unvisited neighbors (continue DFS)
            for neighbor in range(self.size - 1, -1, -1):  # reverse order to simulate stack behavior
                if neighbor not in path and self.edges[last_node][neighbor] != 0:
                    new_path = path + [neighbor]
                    stack.append(new_path)
                    
        return all_paths  

    def bfs_all_paths(self, starting_node: int):
        """
        Perform BS to explore all possible paths, both complete and incomplete.
        
        Parameters:
        starting_node : int - the node from which to start the search
        
        Returns:
        list : A list of paths. Path can be incomplete or complete.
        """
        from collections import deque

        # queue for bfs
        # queue = [[starting_node]]  # with this by hand implementation this algorithm was useless already around n = 10. As I read due to O(n) pop(0) opearation. 
        queue = deque([[starting_node]]) 

        all_paths = []

        while queue:
            # path = queue.pop(0)   # take first element from list (queue)
            path = queue.popleft()
            last_node = path[-1]
            
            # Record the current (incomplete or complete) path and its cost
            all_paths.append(path)

            # If the path includes all cities, check if it can return to the start
            if len(path) == self.size:
                return_to_start_cost = self.edges[last_node][starting_node]   # to make sure that we can return to start from there
                if return_to_start_cost != 0:
                    # Record the completed cycle (path returns to the start)
                    all_paths.append(path + [starting_node])

                continue

            # Explore all unvisited neighbors (continue bfs)
            for neighbor in range(0, self.size):  # normal queue order
                if neighbor not in path and self.edges[last_node][neighbor] != 0:
                    new_path = path + [neighbor]
                    queue.append(new_path)

        return all_paths
    def get_distance_uncomplete_cycle(self,cycle):
        distance = 0
        path_len = len(cycle)
        for index in range(path_len-1):
         # it's in fact len(state_tree) - 1 -> exactly what we need in this approach
            distance += self.edges[cycle[index]][cycle[index+1]]  # we need to extract ith and ith+1 element of our cycle 
        return distance
    
    
    def get_distance(self,cycle):
        distance = 0
        for index in range(self.size):  # it's in fact len(state_tree) - 1 -> exactly what we need in this approach
            distance += self.edges[cycle[index]][cycle[index+1]]  # we need to extract ith and ith+1 element of our cycle 
        return distance
    
    def get_min_distance(self,state_tree):
        """ Requires state tree object from for example dfs or bfs algo, calculate distance only for full paths"""
        
        min_distance = self.max
        shortest_route = []
        for cycle in state_tree:
            if (len(cycle)==self.size + 1):  # we calculate distance only in case of full cycle
                distance = self.get_distance(cycle)
            
                if (distance < min_distance):
                    min_distance = distance
                    shortest_route = cycle

        return (shortest_route,min_distance)
    
    def nn(self,start):
        
        path = [start]
        
        while(len(path)!=self.size):
            dist = maxsize  # initialize distance as maxsize
            node = path[-1]  # get last node from path as a new 'starting node'
            target = -1  # initialize target as a number from outside matrix index
            for neighbour in range(self.size):   # iterate over neighbours of 'new' target                 
                cur_dist = self.edges[node][neighbour] 
                if ((neighbour not in path) & (0 < cur_dist < dist)):    # if path exists and is shorter than previous lower from this starting point 
                    target = neighbour   # store neighbour
                    dist = cur_dist   # store shortest distance 
                    
            if target == -1:    # we got in the dead end 
           
                return path , "Didn't find complete route"
            path.append(target)
        
        if (self.edges[target][start]!=0):    # if we reached there and we can go back to start we found complete circle
            path.append(start)
           
            return path , self.get_distance(path)
        
        # póki co nie testuję nn
      
        return path, "Didn't find complete route - no connection between last node and starting node" 
    
    def heuristic(self,path,heuristic_type):
        
        if (self.size<=len(path)):
            return 0 
        
        else:
            if heuristic_type ==  'admissible':
                # minimum for each unvisited node 
                
                unvisited = [node for node in range(self.size) if node not in path]
                current = path[-1]
                min_edges = []  # Store the minimum outgoing edge for each unvisited node
                for node in unvisited:
                    node_edges = [self.edges[node][j] for j in range(self.size)
                                  if self.edges[node][j]!=0]
                    min_edges.append(min(node_edges))  # Collect minimum for each node
                    # local minimum for edge 
                    # now exrract minimum to make sure that heurisitc is admissible and consistant
                    
                minimium = min(min_edges)
                return minimium * (self.size- len(path))  # adding minimum because we need to go back to start so we can safely add that 
                return minimium * (self.size- len(path) )  # adding minimum because we need to go back to start so we can safely add that            
            elif heuristic_type ==  'inadmissible':
                 # minimum to the neighbour and than straight line to the start
                current = path[-1]
                minimum = maxsize
                for neighbour in range(self.size):
                    if neighbour not in path and self.edges[current][neighbour]!=0:
                        if self.edges[current][neighbour] < minimum:
                            minimum = self.edges[current][neighbour]
                            
                if minimum == maxsize:
                    return 0 
                return minimum * (self.size- len(path))  
                # return minimum * (self.size- len(path)) # because in our queue we don't check distance to start
                
            else:
                raise ValueError("Invalid heuristic type, use 'admissible' or 'inadmissible' eventually None if you want to perform Djikstra instead of A_star")
                
        

    
    
    
    def a_star(self,start,heuristic_type):
        
        pq = PriorityQueue() 
        
        pq.put((0, [start])) # pair, distance path 

        distance , path = pq.get()
        current_node = path[-1]

        while len(path) != self.size:   # because we want to find closed tour in this loop 
            # if self.edges[current_node][start] != 0:
            #     total_distance = distance + self.edges[current_node][start]
            #     complete_path = path + [start]
                
                   
            
            for neighbour in range(self.size):
                if neighbour not in path and self.edges[current_node][neighbour] != 0:
                    # distance = self.get_distance_uncomplete_cycle(path)                       
                    # new_distance = distance + self.edges[current_node][neighbour]
                    
                    # Create new path
                    if ((self.edges[neighbour][start]==0) & (len(path) == self.size-1)):  # skip if we cant make a way to start
                        continue 
                    new_path = path + [neighbour] 
                    
                   
                    to_start = new_path  + [start]
                    new_distance = self.get_distance_uncomplete_cycle(to_start)
                    if heuristic_type:
                        h = self.heuristic(to_start,heuristic_type)
                        new_distance += h
                    
                    pq.put((new_distance,new_path))
            distance , path = pq.get()
            current_node = path[-1]
            # I will add possibility of going back to start there. It will increase time complexity of loop by some constant but will guarantee that we find complete solution in loop
            # if len(path)==self.size:
            #     # try to close 
            #     if self.edges[current_node][start]!=0:
            #         distance += self.edges[current_node][start]
            #         path = path + [start]
            # else: 
            #     distnace,path = pq.get()  # go back one step and continue loop if we couldn't close loop 
            # #useless because it will go back to same solutions anyway 
        # return path,distance
        if self.edges[current_node][start]!=0:
            return path + [start] , distance   # distance is already added/ Just check is it valid and return cycle in same style as in others
        else:
            print("Can't complete tour")
            return path, None
            
    
    def dijkstra(self,start):
        return self.a_star(start,heuristic_type=None)

    def aco(self, alfa =0.9 , beta = 1.5 , decay = .5, Q = 5,steps = 10, ants = None ,best_half_update= True,random_seed = 42,print_info = True):
        
        if random_seed:
            random.seed(random_seed)
        
        # default to number of cities in the problem
        if ants:
            pass
        else:  
            ants = self.size    
        
        # initialize matrix for pheromones storage 
        phero_table = [[1 for _ in range(self.size)] for _ in range(self.size)]
        shortest_distance = maxsize
        def ant_path(start):
            path = [start]
            path_distance = 0
            
            # we will first create list of possibles continuations and their probs 
            while len(path) < self.size: 
                current = path[-1]

                probs = []
                possible_steps = []
                for neighbour in range(self.size):
                    dist = self.edges[current][neighbour]
                    if ((dist != 0)& (neighbour not in path)):
                        possible_steps.append(neighbour)  # we append possible step together with pheromone of it 
                        prob_temp = phero_table[current][neighbour]**alfa / dist **beta # we store distance there 
                        probs.append(prob_temp)

                if len(possible_steps) == 0: 
                    # print("First if fired")
                    return path,maxsize   # we went to dead end 
                next_step = random.choices(possible_steps,weights=probs)[0]    # we are choosing next step, appending distance and direction 
                path_distance += self.edges[current][next_step]
                path.append(next_step)
                current = next_step
            # try to return to start 
            if (self.edges[current][path[0]]!=0):
                path.append(start)
                path_distance += self.edges[current][path[0]] # back to the start 
            else:
                return path, maxsize  # we will divide by very huge number so we will update pheromon by very low value  
            # if we get there we found complet cycle. Let's return path and  distnace of cycle
            return path, path_distance
            
        
        for step in range(steps):
            # we will place ants in order and if ants>self.size then we will start from the 0 index city again 
            results_table = []
            
            for ant in range(ants):
                results_table.append(ant_path(ant % self.size))

            # also let's keep best solution in step for now just for printing it out
            results_table.sort(key = lambda x: x[1])

            # if update only based on top half of solution 
            if best_half_update:
                results_table = results_table[:len(results_table)//2]
            
            
            # now we need to update phero_table according to formula, first of all we can apply decay rate 
            
            for i in range(self.size):
                for j in range(self.size):
                    phero_table[i][j] = phero_table[i][j] * (1 - decay)
        
            # now update based on result table - solutions generated by ants in previous step 
            for route, distance in results_table: 
                for k in range (len(route)-1):
                    r = route[k]
                    s = route[k+1]   # we extract pairs of city on path of this ant
                    # and update pehero_table in cosent with formula
                    phero_table[r][s] += Q / distance   
            if print_info:                      
                print(f"Step: {step} best route distance: {results_table[0][1]}")
        
            # keep best soluton 
            if results_table[0][1] < shortest_distance:
                shortest_distance = results_table[0][1]
                best_solution = results_table[0][0] , results_table[0][1]
        
        
        
        return best_solution

In [163]:
# test = CitiesMap(20,seed=42)  # this will generate route without connection with nn
test = CitiesMap(10,seed=40)
test.calculate_distance(symmetrical=False,connections=0.8);

In [164]:
algos = RouteSearch(test)

In [165]:

paths = algos.dfs_all_paths(0)


In [166]:
paths1 = algos.bfs_all_paths(0)

In [167]:
print("Paths generated by DFS:")
paths

Paths generated by DFS:


[[0],
 [0, 1],
 [0, 1, 3],
 [0, 1, 3, 2],
 [0, 1, 3, 2, 4],
 [0, 1, 3, 2, 4, 5],
 [0, 1, 3, 2, 4, 5, 6],
 [0, 1, 3, 2, 4, 5, 6, 7],
 [0, 1, 3, 2, 4, 5, 6, 7, 8],
 [0, 1, 3, 2, 4, 5, 6, 7, 9],
 [0, 1, 3, 2, 4, 5, 6, 8],
 [0, 1, 3, 2, 4, 5, 6, 8, 7],
 [0, 1, 3, 2, 4, 5, 6, 8, 7, 9],
 [0, 1, 3, 2, 4, 5, 6, 8, 7, 9, 0],
 [0, 1, 3, 2, 4, 5, 6, 9],
 [0, 1, 3, 2, 4, 5, 6, 9, 7],
 [0, 1, 3, 2, 4, 5, 6, 9, 7, 8],
 [0, 1, 3, 2, 4, 5, 7],
 [0, 1, 3, 2, 4, 5, 7, 6],
 [0, 1, 3, 2, 4, 5, 7, 6, 8],
 [0, 1, 3, 2, 4, 5, 7, 6, 9],
 [0, 1, 3, 2, 4, 5, 7, 8],
 [0, 1, 3, 2, 4, 5, 7, 8, 6],
 [0, 1, 3, 2, 4, 5, 7, 8, 6, 9],
 [0, 1, 3, 2, 4, 5, 7, 8, 6, 9, 0],
 [0, 1, 3, 2, 4, 5, 7, 9],
 [0, 1, 3, 2, 4, 5, 7, 9, 6],
 [0, 1, 3, 2, 4, 5, 7, 9, 6, 8],
 [0, 1, 3, 2, 4, 5, 8],
 [0, 1, 3, 2, 4, 5, 8, 6],
 [0, 1, 3, 2, 4, 5, 8, 6, 7],
 [0, 1, 3, 2, 4, 5, 8, 6, 7, 9],
 [0, 1, 3, 2, 4, 5, 8, 6, 7, 9, 0],
 [0, 1, 3, 2, 4, 5, 8, 6, 9],
 [0, 1, 3, 2, 4, 5, 8, 6, 9, 7],
 [0, 1, 3, 2, 4, 5, 8, 7],
 [0, 1, 3, 2, 4, 5, 8, 7,

In [168]:
print("Paths generated by BFS:")
paths1

Paths generated by BFS:


[[0],
 [0, 1],
 [0, 2],
 [0, 3],
 [0, 4],
 [0, 5],
 [0, 6],
 [0, 9],
 [0, 1, 3],
 [0, 1, 4],
 [0, 1, 5],
 [0, 1, 6],
 [0, 1, 7],
 [0, 1, 8],
 [0, 2, 3],
 [0, 2, 4],
 [0, 2, 5],
 [0, 2, 7],
 [0, 2, 8],
 [0, 2, 9],
 [0, 3, 1],
 [0, 3, 2],
 [0, 3, 4],
 [0, 3, 5],
 [0, 3, 7],
 [0, 3, 8],
 [0, 3, 9],
 [0, 4, 1],
 [0, 4, 2],
 [0, 4, 3],
 [0, 4, 5],
 [0, 4, 6],
 [0, 4, 9],
 [0, 5, 1],
 [0, 5, 2],
 [0, 5, 3],
 [0, 5, 4],
 [0, 5, 6],
 [0, 5, 7],
 [0, 5, 8],
 [0, 5, 9],
 [0, 6, 1],
 [0, 6, 4],
 [0, 6, 5],
 [0, 6, 7],
 [0, 6, 8],
 [0, 6, 9],
 [0, 9, 2],
 [0, 9, 3],
 [0, 9, 4],
 [0, 9, 5],
 [0, 9, 6],
 [0, 9, 7],
 [0, 1, 3, 2],
 [0, 1, 3, 4],
 [0, 1, 3, 5],
 [0, 1, 3, 7],
 [0, 1, 3, 8],
 [0, 1, 3, 9],
 [0, 1, 4, 2],
 [0, 1, 4, 3],
 [0, 1, 4, 5],
 [0, 1, 4, 6],
 [0, 1, 4, 9],
 [0, 1, 5, 2],
 [0, 1, 5, 3],
 [0, 1, 5, 4],
 [0, 1, 5, 6],
 [0, 1, 5, 7],
 [0, 1, 5, 8],
 [0, 1, 5, 9],
 [0, 1, 6, 4],
 [0, 1, 6, 5],
 [0, 1, 6, 7],
 [0, 1, 6, 8],
 [0, 1, 6, 9],
 [0, 1, 7, 2],
 [0, 1, 7, 3],
 [0, 1, 7, 5],
 

In [169]:
algos.get_min_distance(paths)   # based on route from dfs or bfs 

([0, 3, 7, 8, 6, 4, 9, 2, 5, 1, 0], 913.782050391848)

In [170]:
algos.edges

[[0.0,
  107.98870311287195,
  137.82234942127494,
  55.96436366117281,
  139.4964157245626,
  76.06576102294646,
  133.9897384130591,
  0,
  0,
  141.088624630053],
 [107.922008876781,
  0.0,
  0,
  112.20913510048992,
  157.52526781440494,
  75.46363362574056,
  114.03074146913191,
  192.2571455109016,
  77.86809359423152,
  0],
 [139.12224840046252,
  0,
  0.0,
  138.07392947258364,
  183.06624484049482,
  84.29116205154607,
  0,
  224.70480635714048,
  116.24353745477639,
  86.83893136145792],
 [55.67414121475068,
  112.19309247899356,
  135.85289838645326,
  0.0,
  192.123814244877,
  58.89830218266058,
  0,
  130.99221350904793,
  123.4514074443868,
  166.63015933497752],
 [139.81863252084824,
  158.0841864324196,
  182.82026692902514,
  192.72249479497717,
  0.0,
  182.03365073524182,
  62.64982043070834,
  0,
  0,
  118.79920033400899],
 [78.39642849007855,
  78.82360052674579,
  84.29116205154607,
  63.85460046073423,
  182.28069014572003,
  0.0,
  162.22284056198745,
  164.93

In [171]:
test = [0, 1, 8, 6, 4, 9, 2, 5, 3, 7]
len(test)

10

In [172]:
algos.edges[7][0]

0

In [173]:
algos.nn(0)

([0, 3, 5, 1, 8, 6, 4, 9, 2, 7],
 "Didn't find complete route - no connection between last node and starting node")

In [174]:
algos.dijkstra(0)

[0, 3, 5, 2, 9, 6, 4, 1, 8]
[0, 3, 5, 2, 9, 4, 6, 1, 8]
[0, 3, 5, 1, 4, 6, 9, 2, 8]
[0, 3, 5, 2, 9, 4, 6, 8, 1]
[0, 1, 8, 6, 4, 9, 2, 5, 3]
[0, 3, 5, 1, 6, 4, 9, 2, 8]
[0, 4, 6, 9, 2, 5, 3, 1, 8]
[0, 3, 1, 8, 6, 4, 9, 2, 5]
[0, 5, 2, 9, 4, 6, 8, 1, 3]
[0, 3, 5, 1, 8, 6, 4, 9, 2]
[0, 3, 1, 5, 2, 9, 4, 6, 8]
[0, 2, 9, 4, 6, 8, 1, 5, 3]
[0, 3, 4, 6, 9, 2, 5, 1, 8]
[0, 4, 6, 9, 2, 3, 5, 1, 8]
[0, 3, 1, 4, 6, 9, 2, 5, 8]
[0, 4, 6, 9, 2, 8, 1, 5, 3]
[0, 5, 1, 8, 6, 4, 9, 2, 3]
[0, 3, 1, 6, 4, 9, 2, 5, 8]
[0, 3, 2, 9, 4, 6, 8, 1, 5]
[0, 1, 4, 6, 9, 2, 5, 3, 8]
[0, 4, 6, 9, 2, 5, 1, 3, 8]
[0, 3, 2, 9, 6, 4, 1, 5, 8]
[0, 5, 3, 2, 9, 6, 4, 1, 8]
[0, 1, 6, 4, 9, 2, 5, 3, 8]
[0, 1, 4, 6, 9, 2, 5, 3, 7]
[0, 4, 6, 9, 2, 5, 1, 3, 7]
[0, 5, 3, 1, 4, 6, 9, 2, 8]
[0, 3, 5, 2, 9, 6, 4, 1, 7]
[0, 3, 2, 9, 4, 6, 1, 5, 8]
[0, 1, 6, 4, 9, 2, 5, 3, 7]
[0, 1, 3, 5, 2, 9, 4, 6, 8]
[0, 5, 3, 2, 9, 4, 6, 1, 8]
[0, 6, 4, 9, 2, 5, 3, 1, 8]
[0, 3, 1, 5, 2, 9, 4, 6, 7]
[0, 3, 8, 6, 4, 9, 2, 5, 1]
[0, 5, 2, 9, 6, 4, 1

([0, 3, 7, 8, 6, 4, 9, 2, 5, 1, 0], 913.782050391848)

In [175]:
algos.a_star(0,heuristic_type='admissible')

[0, 3, 5, 2, 9, 4, 6, 8, 1]
[0, 5, 2, 9, 4, 6, 8, 1, 3]
[0, 3, 5, 1, 8, 6, 4, 9, 2]
[0, 3, 5, 2, 9, 6, 4, 1, 8]
[0, 3, 5, 2, 9, 4, 6, 1, 8]
[0, 5, 1, 8, 6, 4, 9, 2, 3]
[0, 4, 6, 9, 2, 5, 1, 3, 8]
[0, 1, 8, 6, 4, 9, 2, 5, 3]
[0, 2, 9, 4, 6, 8, 1, 5, 3]
[0, 3, 5, 1, 4, 6, 9, 2, 8]
[0, 4, 6, 9, 2, 5, 1, 3, 7]
[0, 3, 2, 9, 4, 6, 8, 1, 5]
[0, 3, 5, 2, 9, 6, 4, 1, 7]
[0, 3, 5, 1, 6, 4, 9, 2, 8]
[0, 3, 1, 8, 6, 4, 9, 2, 5]
[0, 4, 6, 9, 2, 8, 1, 5, 3]
[0, 3, 8, 6, 4, 9, 2, 5, 1]
[0, 5, 2, 9, 6, 4, 1, 3, 8]
[0, 3, 5, 2, 9, 4, 6, 1, 7]
[0, 4, 6, 9, 2, 5, 1, 8, 3]
[0, 4, 6, 9, 2, 5, 3, 1, 8]
[0, 5, 2, 9, 6, 4, 1, 3, 7]
[0, 1, 5, 2, 9, 4, 6, 8, 3]
[0, 2, 9, 6, 4, 1, 5, 3, 8]
[0, 1, 8, 6, 4, 9, 2, 3, 5]
[0, 3, 2, 9, 6, 4, 1, 5, 8]
[0, 1, 4, 6, 9, 2, 5, 3, 8]
[0, 1, 4, 6, 9, 2, 5, 3, 7]
[0, 5, 2, 9, 4, 6, 1, 3, 8]
[0, 2, 9, 6, 4, 1, 5, 3, 7]
[0, 3, 1, 5, 2, 9, 4, 6, 8]
[0, 3, 1, 5, 2, 9, 4, 6, 7]
[0, 5, 3, 2, 9, 4, 6, 8, 1]
[0, 1, 6, 4, 9, 2, 5, 3, 8]
[0, 1, 6, 4, 9, 2, 5, 3, 7]
[0, 3, 1, 4, 6, 9, 2

([0, 3, 7, 8, 6, 4, 9, 2, 5, 1, 0], 913.782050391848)

In [176]:
algos.a_star(0,heuristic_type='inadmissible')

[0, 5, 2, 9, 4, 6, 8, 1, 3]
[0, 5, 1, 8, 6, 4, 9, 2, 3]
[0, 4, 6, 9, 2, 5, 1, 3, 8]
[0, 1, 8, 6, 4, 9, 2, 5, 3]
[0, 4, 6, 9, 2, 5, 1, 3, 7]
[0, 2, 9, 4, 6, 8, 1, 5, 3]
[0, 4, 6, 9, 2, 8, 1, 5, 3]
[0, 5, 2, 9, 6, 4, 1, 3, 8]
[0, 4, 6, 9, 2, 5, 3, 1, 8]
[0, 4, 6, 9, 2, 5, 1, 8, 3]
[0, 5, 2, 9, 6, 4, 1, 3, 7]
[0, 3, 1, 8, 6, 4, 9, 2, 5]
[0, 1, 8, 6, 4, 9, 2, 3, 5]
[0, 5, 2, 9, 4, 6, 1, 3, 8]
[0, 1, 5, 2, 9, 4, 6, 8, 3]
[0, 2, 9, 6, 4, 1, 5, 3, 8]
[0, 2, 9, 6, 4, 1, 5, 3, 7]
[0, 5, 2, 9, 4, 6, 1, 3, 7]
[0, 2, 9, 4, 6, 8, 1, 3, 5]
[0, 1, 4, 6, 9, 2, 5, 3, 8]
[0, 1, 4, 6, 9, 2, 5, 3, 7]
[0, 3, 8, 6, 4, 9, 2, 5, 1]
[0, 5, 1, 4, 6, 9, 2, 3, 8]
[0, 5, 2, 9, 6, 4, 1, 8, 3]
[0, 1, 6, 4, 9, 2, 5, 3, 8]
[0, 1, 6, 4, 9, 2, 5, 3, 7]
[0, 5, 1, 6, 4, 9, 2, 3, 8]
[0, 5, 1, 4, 6, 9, 2, 3, 7]
[0, 2, 9, 4, 6, 1, 5, 3, 8]
[0, 2, 9, 4, 6, 1, 5, 3, 7]
[0, 6, 4, 9, 2, 5, 1, 3, 8]
[0, 4, 6, 9, 2, 8, 1, 3, 5]
[0, 4, 6, 9, 2, 5, 8, 1, 3]
[0, 1, 5, 2, 9, 6, 4, 3, 8]
[0, 5, 2, 9, 4, 6, 1, 8, 3]
[0, 5, 1, 6, 4, 9, 2

([0, 3, 7, 8, 6, 4, 9, 2, 5, 1, 0], 913.782050391848)

In [90]:
algos.aco(steps=10,decay=0.9,Q=10,best_half_update=True,random_seed=42,print_info=False)

([6, 4, 1, 8, 7, 3, 0, 5, 2, 9, 6], 935.9838626919936)

In [16]:
import pandas as pd 
import time 
def compare_algorithms(n , symmetrical , conenctions,seed, brute_force):
    """Not recommended to set brute force True for n > 12/13 """
    city_map = CitiesMap(n=n,seed=seed)
    city_map.calculate_distance(symmetrical=symmetrical,connections=conenctions)
    algos_space = RouteSearch(city_map)
    algo_name = []
    path = []
    shortest_path = []
    timer = []
    if brute_force:
        # brute forceprint("")
         print("DFS")
         start = time.time()
         
         paths_ = algos_space.dfs_all_paths(0)
         dfs_distance = algos_space.get_min_distance(paths_)   # based on route from dfs or bfs 
         end = time.time()
         length = end - start
         algo_name.append("DFS")
         path.append(dfs_distance[0])
         shortest_path.append(dfs_distance[1])
         timer.append(length)
         
         print("BFS")
         start = time.time()
         
         paths_1 = algos_space.bfs_all_paths(0)
         bfs_distance = algos_space.get_min_distance(paths_1)   # based on route from dfs or bfs 
         end = time.time()
         length = end - start
         algo_name.append("BFS")
         path.append(bfs_distance[0])
         shortest_path.append(bfs_distance[1])
         timer.append(length)
         
         
         # nn 
        

         
         print("NN")
         start = time.time()
         nn_distance = algos_space.nn(0)
         end = time.time()
         length = end - start
         algo_name.append("Nearest Neighbour")
         path.append(nn_distance[0])
         shortest_path.append(nn_distance[1])
         timer.append(length)
         
         # djikstra will be calculated for n<= 14 ~ 6 minutes
         if n <= 15:
            print("Djikstra")
            start = time.time()
            djisktra_distance = algos_space.dijkstra(0)
            end = time.time()
            length = end - start
            length = end - start
            algo_name.append("Djikstra")
            path.append(djisktra_distance[0])
            shortest_path.append(djisktra_distance[1])
            timer.append(length)
         
         # a _start admmissible heuristic 
         print("A*adm")
         start = time.time()
         a_star_adm = algos_space.a_star(0,heuristic_type="admissible")
         end = time.time()
         length = end - start
         algo_name.append("A* admissible heuristic")
         path.append(a_star_adm[0])
         shortest_path.append(a_star_adm[1])
         timer.append(length)
          # a _start inadmmissible heuristic 
          
         if n<=18:
            print("A*inadm")
            start = time.time()
            a_star_inadm = algos_space.a_star(0,heuristic_type="inadmissible")
            end = time.time()
            length = end - start
            algo_name.append("A* inadmissible heuristic")
            path.append(a_star_inadm[0])
            shortest_path.append(a_star_inadm[1])
            timer.append(length)
            
         # aco 
         print("Aco")
         start = time.time()
         aco_distance = algos_space.aco(steps=200,decay=0.9,Q=10,best_half_update=True,random_seed=42,print_info=False)
         end = time.time()
         length = end - start
         algo_name.append("ACO")
         path.append(aco_distance[0])
         shortest_path.append(aco_distance[1])
         timer.append(length)
         
    else:
        # nn 
        
         
         print("NN")
         start = time.time()
         nn_distance = algos_space.nn(0)
         end = time.time()
         length = end - start
         algo_name.append("Nearest Neighbour")
         path.append(nn_distance[0])
         shortest_path.append(nn_distance[1])
         timer.append(length)
         
        # Djikstra
         if n <= 15:        
            print("Djikstra")        
            start = time.time()
            
            djisktra_distance = algos_space.dijkstra(0)
            end = time.time()
            length = end - start
            algo_name.append("Djikstra")
            path.append(djisktra_distance[0])
            shortest_path.append(djisktra_distance[1])
            timer.append(length)
         
         # a _start admmissible heuristic 
         print("A*adm")
         start = time.time()
         a_star_adm = algos_space.a_star(0,heuristic_type="admissible")
         end = time.time()
         length = end - start
         algo_name.append("A* admissible heuristic")
         path.append(a_star_adm[0])
         shortest_path.append(a_star_adm[1])
         timer.append(length)
          # a _start inadmmissible heuristic 
         if n<=18:
            print("A*inadm")
            start = time.time()
            a_star_inadm = algos_space.a_star(0,heuristic_type="inadmissible")
            end = time.time()
            length = end - start
            algo_name.append("A* inadmissible heuristic")
            path.append(a_star_inadm[0])
            shortest_path.append(a_star_inadm[1])
            timer.append(length)
         
         # aco 
         print("aco")        
         start = time.time()
         aco_distance = algos_space.aco(steps=200,decay=0.9,Q=10,best_half_update=True,random_seed=42,print_info=False)
         end = time.time()
         length = end - start
         algo_name.append("ACO")
         path.append(aco_distance[0])
         shortest_path.append(aco_distance[1])
         timer.append(length)
    df = pd.DataFrame(data = {"Name of algo":algo_name,"Path of solution":path,"Distance of solution":shortest_path,"Time of calculations":timer})
    return df 

In [17]:
df = compare_algorithms(n = 10 , symmetrical= True , conenctions= 1 , brute_force= True, seed = 84);


DFS
BFS
NN
Djikstra
A*adm
A*inadm
Aco


In [18]:
df

,Name of algo,Path of solution,Distance of solution,Time of calculations
0,DFS,"[0, 3, 4, 6, 1, 9, 2, 5, 8, 7, 0]",563.796152,1.009965
1,BFS,"[0, 3, 4, 6, 1, 9, 2, 5, 8, 7, 0]",563.796152,0.758104
2,Nearest Neighbour,"[0, 3, 4, 7, 8, 9, 5, 2, 1, 6, 0]",575.465898,0.000000
3,Djikstra,"[0, 3, 4, 6, 1, 9, 2, 5, 8, 7, 0]",563.796152,0.447565
4,A* admissible heuristic,"[0, 7, 8, 5, 2, 9, 1, 6, 4, 3, 0]",563.796152,0.229154
5,A* inadmissible heuristic,"[0, 7, 8, 5, 2, 9, 1, 6, 4, 3, 0]",563.796152,0.045999
6,ACO,"[4, 3, 0, 6, 1, 9, 5, 2, 8, 7, 4]",573.394926,0.031317


In [19]:
df1 = compare_algorithms(n = 8 , symmetrical= False , conenctions= 1 , brute_force= True,seed= 6);
df1

DFS
BFS
NN
Djikstra
A*adm
A*inadm
Aco


,Name of algo,Path of solution,Distance of solution,Time of calculations
0,DFS,"[0, 6, 2, 3, 7, 4, 5, 1, 0]",566.096296,0.008001
1,BFS,"[0, 6, 2, 3, 7, 4, 5, 1, 0]",566.096296,0.007000
2,Nearest Neighbour,"[0, 6, 2, 3, 7, 4, 5, 1, 0]",566.096296,0.000000
3,Djikstra,"[0, 6, 2, 3, 7, 4, 5, 1, 0]",566.096296,0.004998
4,A* admissible heuristic,"[0, 6, 2, 3, 7, 4, 5, 1, 0]",566.096296,0.007002
5,A* inadmissible heuristic,"[0, 6, 2, 3, 7, 4, 5, 1, 0]",566.096296,0.001000
6,ACO,"[1, 0, 6, 2, 3, 7, 4, 5, 1]",566.096296,0.018000


In [23]:
df2 = compare_algorithms(n = 12 , symmetrical= True , conenctions= 0.8 , brute_force= True, seed = 12);
df2 

DFS
BFS
NN
Djikstra
Can't complete tour
A*adm
Can't complete tour
A*inadm
Can't complete tour
Aco


,Name of algo,Path of solution,Distance of solution,Time of calculations
0,DFS,"[0, 11, 8, 2, 7, 3, 1, 10, 6, 4, 5, 9, 0]",691.607929,7.871579
1,BFS,"[0, 11, 8, 2, 7, 3, 1, 10, 6, 4, 5, 9, 0]",691.607929,6.590830
2,Nearest Neighbour,"[0, 11, 8, 3, 2, 7, 1, 10, 6, 4, 5, 9, 0]",711.280430,0.000000
3,Djikstra,"[0, 9, 5, 4, 6, 10, 1, 3, 7, 2, 11, 8]",NaN,1.094112
4,A* admissible heuristic,"[0, 11, 9, 5, 4, 6, 10, 1, 3, 7, 2, 8]",NaN,1.276708
5,A* inadmissible heuristic,"[0, 11, 6, 10, 3, 2, 7, 1, 9, 5, 4, 8]",NaN,0.001999
6,ACO,"[4, 1, 10, 6, 2, 3, 7, 8, 11, 0, 9, 5, 4]",715.045247,0.049944


In [21]:
df122 = compare_algorithms(n = 12 , symmetrical= True , conenctions= 1 , brute_force= True, seed = 2000);
df122

DFS
BFS
NN
Djikstra
A*adm
A*inadm
Aco


,Name of algo,Path of solution,Distance of solution,Time of calculations
0,DFS,"[0, 2, 6, 5, 1, 8, 7, 3, 9, 11, 4, 10, 0]",760.953510,120.850050
1,BFS,"[0, 2, 6, 5, 1, 8, 7, 3, 9, 11, 4, 10, 0]",760.953510,117.958302
2,Nearest Neighbour,"[0, 6, 2, 5, 3, 7, 1, 8, 11, 4, 10, 9, 0]",800.242717,0.000000
3,Djikstra,"[0, 2, 6, 5, 1, 8, 7, 3, 9, 11, 4, 10, 0]",760.953510,1.985786
4,A* admissible heuristic,"[0, 2, 6, 5, 1, 8, 7, 3, 9, 11, 4, 10, 0]",760.953510,2.508595
5,A* inadmissible heuristic,"[0, 6, 9, 4, 10, 11, 7, 3, 1, 8, 5, 2, 0]",877.533586,0.001000
6,ACO,"[10, 4, 11, 9, 3, 7, 8, 1, 5, 2, 6, 0, 10]",777.753778,0.049509


In [177]:
df122 = compare_algorithms(n = 12 , symmetrical= True , conenctions= 1 , brute_force= True, seed = 2000);
df122  

DFS
BFS
NN
Djikstra
A*adm
A*inadm
Aco


,Name of algo,Path of solution,Distance of solution,Time of calculations
0,DFS,"[0, 2, 6, 5, 1, 8, 7, 3, 9, 11, 4, 10, 0]",760.953510,117.005266
1,BFS,"[0, 2, 6, 5, 1, 8, 7, 3, 9, 11, 4, 10, 0]",760.953510,111.221223
2,Nearest Neighbour,"[0, 6, 2, 5, 3, 7, 1, 8, 11, 4, 10, 9, 0]",800.242717,0.000000
3,Djikstra,"[0, 2, 6, 5, 1, 8, 7, 3, 9, 11, 4, 10, 0]",760.953510,2.056826
4,A* admissible heuristic,"[0, 2, 6, 5, 1, 8, 7, 3, 9, 11, 4, 10, 0]",760.953510,2.607045
5,A* inadmissible heuristic,"[0, 6, 9, 4, 10, 11, 7, 3, 1, 8, 5, 2, 0]",877.533586,0.000000
6,ACO,"[10, 4, 11, 9, 3, 7, 8, 1, 5, 2, 6, 0, 10]",777.753778,0.053003


In [ ]:
df122 = compare_algorithms(n = 13 , symmetrical= True , conenctions= 1 , brute_force= True, seed = 997);
df122

DFS


In [23]:
df12211 = compare_algorithms(n = 12 , symmetrical= True , conenctions= 1 , brute_force= True, seed = 321132);
df12211

DFS
BFS
NN
Djikstra
A*adm
A*inadm
Aco


,Name of algo,Path of solution,Distance of solution,Time of calculations
0,DFS,"[0, 7, 11, 9, 8, 10, 5, 1, 2, 4, 3, 6, 0]",686.295484,120.007325
1,BFS,"[0, 7, 11, 9, 8, 10, 5, 1, 2, 4, 3, 6, 0]",686.295484,108.342500
2,Nearest Neighbour,"[0, 6, 3, 4, 8, 9, 11, 7, 10, 5, 1, 2, 0]",771.212153,0.000000
3,Djikstra,"[0, 6, 3, 4, 2, 1, 5, 10, 8, 9, 11, 7, 0]",686.295484,0.967757
4,A* admissible heuristic,"[0, 6, 3, 4, 2, 1, 5, 10, 8, 9, 11, 7, 0]",686.295484,0.466976
5,A* inadmissible heuristic,"[0, 6, 3, 4, 7, 11, 9, 8, 10, 5, 1, 2, 0]",774.922840,0.191996
6,ACO,"[8, 10, 5, 1, 2, 6, 3, 4, 0, 7, 11, 9, 8]",719.306159,0.052051


In [74]:
df3 = compare_algorithms(n = 12 , symmetrical= False , conenctions= 0.8 , brute_force= True, seed = 12);
df3 

DFS
BFS
NN
Djikstra
Can't complete tour
A*adm
Can't complete tour
A*inadm
Can't complete tour
Aco


,Name of algo,Path of solution,Distance of solution,Time of calculations
0,DFS,"[0, 11, 8, 2, 7, 3, 1, 10, 6, 4, 5, 9, 0]",690.328294,10.772492
1,BFS,"[0, 11, 8, 2, 7, 3, 1, 10, 6, 4, 5, 9, 0]",690.328294,9.641513
2,Nearest Neighbour,"[0, 11, 8, 3, 2, 7, 1, 10, 6, 4, 5, 9, 0]",713.368555,0.000000
3,Djikstra,None,NaN,1.135716
4,A* admissible heuristic,None,NaN,3.402544
5,A* inadmissible heuristic,None,NaN,0.000998
6,ACO,"[5, 9, 0, 11, 8, 7, 3, 2, 6, 10, 1, 4, 5]",715.932688,0.051999


In [75]:
df44 = compare_algorithms(n = 12 , symmetrical= True , conenctions= 0.8 , brute_force= True, seed = 144);
df44

DFS
BFS
NN
Djikstra
A*adm
A*inadm
Aco


,Name of algo,Path of solution,Distance of solution,Time of calculations
0,DFS,"[0, 5, 6, 10, 2, 4, 7, 3, 8, 11, 1, 9, 0]",629.171337,7.592494
1,BFS,"[0, 5, 6, 10, 2, 4, 7, 3, 8, 11, 1, 9, 0]",629.171337,8.296129
2,Nearest Neighbour,"[0, 5, 6, 10, 4, 2, 3, 8, 1, 11, 7, 9, 0]",686.923263,0.000000
3,Djikstra,"[0, 5, 6, 10, 2, 4, 7, 3, 8, 11, 1, 9, 0]",629.171337,0.689204
4,A* admissible heuristic,"[0, 5, 6, 10, 2, 4, 7, 3, 8, 11, 1, 9, 0]",629.171337,0.342155
5,A* inadmissible heuristic,"[0, 9, 1, 11, 8, 3, 7, 4, 2, 10, 6, 5, 0]",629.171337,0.099508
6,ACO,"[2, 4, 7, 3, 8, 11, 1, 9, 0, 5, 6, 10, 2]",629.171337,0.051513


In [76]:
df44 = compare_algorithms(n = 12 , symmetrical= True , conenctions= 0.8 , brute_force= True, seed = 13);
df44

DFS
BFS
NN
Djikstra
Can't complete tour
A*adm
Can't complete tour
A*inadm
Aco


,Name of algo,Path of solution,Distance of solution,Time of calculations
0,DFS,"[0, 2, 11, 4, 6, 1, 7, 3, 8, 10, 9, 5, 0]",797.428012,7.987522
1,BFS,"[0, 2, 11, 4, 6, 1, 7, 3, 8, 10, 9, 5, 0]",797.428012,6.359271
2,Nearest Neighbour,"[0, 5, 9, 10, 1, 6, 3, 7, 11, 2, 8]",Didn't find complete route,0.000000
3,Djikstra,None,None,3.668121
4,A* admissible heuristic,None,None,1.493132
5,A* inadmissible heuristic,"[0, 5, 9, 10, 8, 3, 7, 1, 6, 4, 11, 2, 0]",797.428012,0.097511
6,ACO,"[6, 4, 11, 2, 0, 5, 10, 9, 7, 3, 8, 1, 6]",814.544341,0.051002


In [77]:
df4444 = compare_algorithms(n = 12 , symmetrical= True , conenctions= 0.8 , brute_force= True, seed = 124142);
df4444

DFS
BFS
NN
Djikstra
Can't complete tour
A*adm
Can't complete tour
A*inadm
Can't complete tour
Aco


,Name of algo,Path of solution,Distance of solution,Time of calculations
0,DFS,"[0, 11, 9, 10, 6, 5, 1, 2, 3, 7, 4, 8, 0]",778.084011,7.728774
1,BFS,"[0, 11, 9, 10, 6, 5, 1, 2, 3, 7, 4, 8, 0]",778.084011,8.325640
2,Nearest Neighbour,"[0, 8, 4, 11, 9, 10, 6, 5, 7, 3, 1, 2, 0]",821.832583,0.000000
3,Djikstra,None,NaN,3.779485
4,A* admissible heuristic,None,NaN,2.045950
5,A* inadmissible heuristic,None,NaN,0.655535
6,ACO,"[2, 3, 11, 8, 0, 4, 7, 9, 10, 6, 5, 1, 2]",797.726475,0.052002


In [57]:
df5444 = compare_algorithms(n = 12 , symmetrical= True , conenctions= 0.8 , brute_force= True, seed = 999);
df5444

DFS
BFS
NN
Djikstra
Can't complete tour
A*adm
Can't complete tour
A*inadm
Aco


,Name of algo,Path of solution,Distance of solution,Time of calculations
0,DFS,"[0, 5, 6, 8, 9, 2, 7, 3, 1, 11, 10, 4, 0]",835.973474,8.708040
1,BFS,"[0, 5, 6, 8, 9, 2, 7, 3, 1, 11, 10, 4, 0]",835.973474,7.839061
2,Nearest Neighbour,"[0, 4, 10, 11, 1, 3, 8, 9, 2, 7, 5, 6, 0]",905.286293,0.000000
3,Djikstra,None,NaN,2.319686
4,A* admissible heuristic,None,NaN,2.211593
5,A* inadmissible heuristic,"[0, 4, 10, 11, 1, 3, 7, 2, 9, 8, 6, 5, 0]",835.973474,0.431244
6,ACO,"[10, 4, 0, 5, 6, 8, 9, 2, 7, 1, 3, 11, 10]",843.490324,0.052000


In [28]:
df15 = compare_algorithms(n = 15 , symmetrical= True , conenctions= 1 , brute_force= False, seed = 12);
df15

NN
Djikstra
A*adm
A*inadm
aco


,Name of algo,Path of solution,Distance of solution,Time of calculations
0,Nearest Neighbour,"[0, 10, 14, 1, 6, 2, 3, 9, 13, 7, 12, 4, 11, 5...",774.419818,0.000000
1,Djikstra,"[0, 8, 5, 11, 4, 14, 10, 3, 9, 2, 6, 1, 13, 7,...",831.791105,759.369339
2,A* admissible heuristic,"[0, 8, 5, 11, 4, 14, 10, 3, 9, 2, 6, 1, 13, 7,...",831.791105,158.405805
3,A* inadmissible heuristic,"[0, 10, 3, 9, 2, 6, 1, 13, 7, 14, 8, 5, 11, 4,...",889.144979,2.044479
4,ACO,"[12, 7, 13, 1, 6, 2, 9, 3, 10, 14, 0, 8, 5, 11...",750.399602,0.108022


In [ ]:
df16 = compare_algorithms(n = 16 , symmetrical= True , conenctions= 1 , brute_force= False  , seed = 42)

NN
A*adm


Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x000002403D8C6510>>
Traceback (most recent call last):
  File "C:\Users\akopa\AppData\Roaming\Python\Python312\site-packages\ipykernel\ipkernel.py", line 790, in _clean_thread_parent_frames
    active_threads = {thread.ident for thread in threading.enumerate()}
                                                 ^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\akopa\AppData\Local\Programs\Python\Python312\Lib\threading.py", line 1544, in enumerate
    return list(_active.values()) + list(_limbo.values())
           ^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt: 


In [27]:
df16

,Name of algo,Path of solution,Distance of solution,Time of calculations
0,Nearest Neighbour,"[0, 7, 9, 13, 15, 14, 3, 12, 10, 5, 6, 2, 8, 1...",1081.908062,0.000000
1,A* admissible heuristic,"[0, 13, 7, 9, 15, 14, 6, 3, 12, 10, 5, 1, 2, 1...",922.844819,8.305516
2,A* inadmissible heuristic,"[0, 13, 7, 4, 11, 1, 10, 5, 6, 3, 12, 2, 8, 14...",947.021476,71.980303
3,ACO,"[9, 15, 14, 6, 5, 12, 3, 10, 1, 2, 8, 11, 4, 1...",909.530165,0.116580


In [78]:
df161 = compare_algorithms(n = 16 , symmetrical= True , conenctions= 0.8 , brute_force= False  , seed = 42)
df161

NN
A*adm
Can't complete tour
A*inadm
Can't complete tour
aco


,Name of algo,Path of solution,Distance of solution,Time of calculations
0,Nearest Neighbour,"[0, 7, 9, 13, 15, 14, 3, 12, 10, 5, 6, 2, 8, 1...",1081.908062,0.000000
1,A* admissible heuristic,None,NaN,78.011403
2,A* inadmissible heuristic,None,NaN,9.450478
3,ACO,"[8, 2, 3, 12, 10, 1, 5, 6, 14, 15, 9, 13, 7, 0...",919.142416,0.115395


In [29]:
df181 = compare_algorithms(n = 18 , symmetrical= True , conenctions= 0.8 , brute_force= False  , seed = 42)
df181

NN
A*adm
A*inadm
aco


,Name of algo,Path of solution,Distance of solution,Time of calculations
0,Nearest Neighbour,"[0, 5, 11, 9, 13, 3, 12, 14, 7, 4, 6, 2, 17, 1...",1142.032216,0.000000
1,A* admissible heuristic,"[0, 5, 11, 9, 13, 3, 4, 6, 2, 12, 14, 7, 16, 1...",1081.483908,39.611751
2,A* inadmissible heuristic,"[0, 13, 3, 12, 14, 7, 4, 6, 2, 5, 11, 1, 15, 8...",1078.435228,12.044655
3,ACO,"[8, 1, 15, 11, 5, 0, 13, 9, 2, 6, 4, 3, 12, 14...",996.523486,0.155007


In [ ]:
df20 = compare_algorithms(n = 20 , symmetrical= True , conenctions= 1 , brute_force= False  , seed = 42)
df20

NN
A*adm
aco


,Name of algo,Path of solution,Distance of solution,Time of calculations
0,Nearest Neighbour,"[0, 12, 10, 4, 8, 9, 6, 11, 14, 1, 7, 3, 5, 15...",1117.296975,0.000000
1,A* admissible heuristic,"[0, 12, 10, 4, 8, 2, 6, 11, 14, 1, 7, 3, 5, 16...",921.443636,630.448307
2,ACO,"[16, 18, 15, 13, 17, 19, 11, 6, 4, 10, 0, 12, ...",942.330748,0.299773


: 